In [2]:
import os
import pdb
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import random
import ipdb

In [42]:
class KITTIDataset(Dataset):
    def __init__(self, root_dir, debug=False):
        """
        Args:
            root_dir (string): Directory with all the point clouds.
        """
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
            
        if debug:
            random_sample = random.randint(1, 7400)
            self.files = [self.files[random_sample]]
        

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        point_cloud_path = os.path.join(self.root_dir, self.files[idx])
        point_cloud = self.load_point_cloud_from_bin(point_cloud_path)  # Assuming point clouds are stored as .pt files
        return point_cloud
    
    def load_point_cloud_from_bin(self, root_dir):
        with open(bin_path, 'rb') as f:
            content = f.read()
            point_cloud = np.frombuffer(content, dtype=np.float32)
            point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
        return torch.from_numpy(point_cloud)
        

train_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced'
test_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/testing/velodyne_reduced'
# TODO: Add label dir

train_set = KITTIDataset(root_dir=train_dir, debug=False)
test_set = KITTIDataset(root_dir=test_dir, debug=False)

batched_train_set = DataLoader(train_set, batch_size=4, shuffle=False)
batched_test_set = DataLoader(train_set, batch_size=4, shuffle=False)




'/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced/000039.bin'

In [4]:

class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PillarFeatureNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Input x is of shape (D, P, N)
        # Convert it to (P, D, N) for 1x1 convolution
        x = x.permute(1, 0, 2)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # Max pooling operation over the points' dimension
        x, _ = torch.max(x, dim=2)  # Output shape: (P, C)
        return x.T  # Output shape: (C, P)



class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
        self.num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        self.num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]

        
        # Using numpy's digitize to find the interval/bin each point belongs to.
        # TODO: Get rid of these unnecessary copies of tensors
        self.x_indices = torch.tensor(np.digitize(points[:, 0], np.linspace(self.x_min, self.x_max, self.num_x_pillars))) - 1
        self.y_indices = torch.tensor(np.digitize(points[:, 1], np.linspace(self.y_min, self.y_max, self.num_y_pillars))) - 1

        # TODO: Calculate pillar x-y center:
        pillar_x_center = self.x_indices * self.pillar_size[0] + self.pillar_size[0] / 2.0
        pillar_y_center = self.y_indices * self.pillar_size[1] + self.pillar_size[1] / 2.0 

        pillars = torch.zeros((self.num_x_pillars, self.num_y_pillars, self.max_points_per_pillar, self.aug_dim))
        
        # Count how many points are in each pillar to ensure we don't exceed `max_points_per_pillar`
        count = torch.zeros((self.num_x_pillars, self.num_y_pillars), dtype=torch.long)
        
        # TODO: Store points in the pillars in a vectorized way filling the pillars tensor:        
        for i in range(points.shape[0]):
            x_ind = self.x_indices[i]
            y_ind = self.y_indices[i]
            
            if count[x_ind, y_ind] < self.max_points_per_pillar:
                # Compute x_c, y_c and z_c
                x_c = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0) - points[i, 0]
                y_c = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0) - points[i, 1]
                z_c = (self.z_min + self.z_max) / 2 - points[i, 2] # assuming the z-center is the midpoint
                
                # Calculate pillar center
                x_pillar_center = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0)
                y_pillar_center = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0)
                
                # Add original x, y, and z coordinates, then x_c, y_c, z_c
                pillars[x_ind, y_ind, count[x_ind, y_ind], :3] = points[i, :3]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c]) # Sus
                pillars[x_ind, y_ind, count[x_ind, y_ind], 6] = x_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 0]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 7] = y_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 1]
                
                count[x_ind, y_ind] += 1
        
        # Zero-padding if too few point, random sampling if too many points:
        for i in range(self.num_x_pillars):
            for j in range(self.num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars
        

    

            
# Pillarization testing:

# TODO: Get a random sample:
def load_point_cloud_from_bin(bin_path):
    with open(bin_path, 'rb') as f:
        content = f.read()
        point_cloud = np.frombuffer(content, dtype=np.float32)
        point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
    
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]
    
    return point_cloud

# Example usage
random_sample = random.randint(1, 7400)
bin_path = os.path.join(train_dir, '000039.bin')
point_cloud = load_point_cloud_from_bin(bin_path)

# TODO: Pillarize the sample:
D = 9 # Augmented dimension
N = 100 # Max number of points per pillar

pillarizer = Pillarization(aug_dim=D, x_min=-40.0, x_max=40.0, y_min=-25.0, y_max=25.0, 
           z_min=-3, z_max=3, pillar_size=(0.5, 0.5), max_points_per_pillar=N)

pillars = pillarizer.make_pillars(torch.from_numpy(point_cloud)) 


# Apply linear activation, batchnorm and ReLU for feature extraction from pillars tensor:
feature_extractor = PillarFeatureNet(D, 64)
features = feature_extractor(pillars)     

# Generate pseudo-image
pseudo_image = torch.zeros(features.shape[0], pillarizer.num_y_pillars, pillarizer.num_x_pillars) # Empty canvas


# Scatter the features back to their original pillar locations
for i in range(features.shape[1]):
    x_ind = pillarizer.x_indices[i].long()
    y_ind = pillarizer.y_indices[i].long()
    pseudo_image[:, y_ind, x_ind] = features[:, i]



/tmp/ipykernel_90080/2119310883.py:134: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  pillars = pillarizer.make_pillars(torch.from_numpy(point_cloud))


In [5]:
pseudo_image.size()

torch.Size([64, 100, 160])

In [36]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, L, stride):
        super(Block, self).__init__()
        layers = []
        # First layer with the specified stride
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1))
        #layers.append(nn.BatchNorm2d(out_channels)) # TODO -> Uncomment and batch
        layers.append(nn.ReLU(inplace=True))
        
        # Subsequent layers with stride 1
        for _ in range(1, L):
            layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
            #layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels, stride, output_padding):
        super(UpSample, self).__init__()
        # Assuming stride_out is always half of stride_in based on the diagram
        self.up = nn.Sequential(
            # nn.Upsample(scale_factor=gain),
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, output_padding=output_padding),
            # nn.BatchNorm2d(out_channels), # TODO> Uncomment and batch
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.up(x)

class BackBone(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BackBone, self).__init__()

        # Define blocks with arbitrary L for now (can be tuned based on requirements)
        self.block1 = Block(in_channels, out_channels*2, L=3, stride=1)
        self.block2 = Block(out_channels*2, out_channels*2, L=3, stride=2)
        self.block3 = Block(out_channels*2, out_channels*2, L=3, stride=2)
        

        # Define upsampling layers        
        self.up1 = UpSample(out_channels*2, out_channels*2, stride=1, output_padding=0)
        self.up2 = UpSample(out_channels*2, out_channels*2, stride=2, output_padding=1)
        self.up3 = UpSample(out_channels*2, out_channels*2, stride=4, output_padding=3)
        
    def forward(self, x):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.block3(x2)
        x2.size()
        # Upsample and concatenate
        up_x1 = self.up1(x1)   
        up_x2 = self.up2(x2)
        up_x3 = self.up3(x3)     
        concat_features = torch.cat([up_x1, up_x2, up_x3], dim=0)
        
        return concat_features

    
backbone = BackBone(in_channels=64, out_channels=64)
backbone_features = backbone(pseudo_image)


torch.Size([384, 100, 160])

In [47]:
pseudo_image.size()

torch.Size([64, 100, 160])